In [1]:
!pip install accelerate bitsandbytes peft -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.2/333.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 20.1 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.34.2
    Uninstalling accelerate-0.34.2:
      Successfully uninstalled accelerate-0.34.2


In [2]:
import os
import time
import copy
from dataclasses import dataclass

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F

from datasets import Dataset
from transformers import (
    BitsAndBytesConfig,
    Gemma2PreTrainedModel,
    Gemma2Model,
    Gemma2ForSequenceClassification,
    GemmaTokenizerFast,
    Gemma2Config,
    PreTrainedTokenizerBase, 
    EvalPrediction,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    TrainerCallback,
    get_scheduler
)
from bitsandbytes.optim import AdamW8bit

from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from sklearn.metrics import log_loss, roc_auc_score, f1_score
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/torch/cuda/__init__.py:230: UserWarning: 
NVIDIA L4 with CUDA capability sm_89 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_60 sm_70 sm_75 compute_70 compute_75.
If you want to use the NVIDIA L4 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(


In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2,3'

VER = 'l4-2nd-try' # version of exp
USE_DEOTT_TOKENIZER = True  # https://www.kaggle.com/competitions/lmsys-chatbot-arena/discussion/527596

HAVE_GOOD_GPU = True

In [ ]:
os.environ['WANDB_API_KEY'] = 'no please'
os.environ['WANDB_PROJECT'] = 'FIC Gemma2-9b'
os.environ['WANDB_NOTES'] = f'FIC Gemma2-9b LoRA Training VER-{VER}'
os.environ['WANDB_NAME'] = f'ft-gemma2-fic-ver-{VER}'

In [5]:
@dataclass
class Config:
    output_dir: str = 'output'
    checkpoint: str = 'unsloth/gemma-2-9b-it-bnb-4bit'
    num_labels: int = 2
    hdim: int = 3584
    max_length: int = 1024
    optim_type: str = 'adamw_8bit'
    per_device_train_batch_size: int = 2  # 4
    gradient_accumulation_steps: int = 4  # 2
    per_device_eval_batch_size: int = 4  # 8
    n_epochs: int = 1  # 2
    freeze_layers: int = 0 if HAVE_GOOD_GPU else 16
    lr: float = 2e-4  # 1e-5, 5e-5, 2e-5
    warmup_steps: int = 60
    lora_r: int = 64  # 128, 1024
    lora_alpha: float = 16 # lora_r * 2
    lora_dropout: float = 0.05
    lora_bias: str = 'none'
    
config = Config() # mb try to set learning rate of the backbone = alpha/rank * head_LRя

In [6]:
training_args = TrainingArguments(
    output_dir=f'output-{VER}',
    overwrite_output_dir=True,
    report_to='wandb',
    num_train_epochs=config.n_epochs,
    per_device_train_batch_size=config.per_device_train_batch_size,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    per_device_eval_batch_size=config.per_device_eval_batch_size,
    logging_steps=1,
    eval_strategy='epoch',
    save_strategy='steps',
    save_steps=100,
    #optim=config.optim_type,
    fp16=True,
    #bf16=False,
    learning_rate=config.lr,
    #warmup_steps=config.warmup_steps,
    #lr_scheduler_type='cosine',  # 'cosine' or 'linear' or 'constant' (default is 'linear')
)

In [7]:
lora_config = LoraConfig(
    r=config.lora_r,
    lora_alpha=config.lora_alpha,
    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj'],
    layers_to_transform=[i for i in range(42) if i >= config.freeze_layers],
    lora_dropout=config.lora_dropout,
    bias=config.lora_bias,
    task_type=TaskType.SEQ_CLS,
)

In [8]:
tokenizer = GemmaTokenizerFast.from_pretrained(config.checkpoint)
tokenizer.add_eos_token = True
tokenizer.padding_side = 'right'

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [9]:
class CustomGemma2ForSequenceClassification(Gemma2ForSequenceClassification):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.model = Gemma2Model(config)
        self.score = torch.nn.Sequential(
            torch.nn.Dropout(0.1),
            torch.nn.Linear(config.hidden_size, config.hidden_size // 2),
            torch.nn.Dropout(0.1),
            torch.nn.GELU(),
            torch.nn.Linear(config.hidden_size // 2, config.num_labels),
        )
        self.post_init()

    def get_input_embeddings(self):
        return self.model.embed_tokens

    def set_input_embeddings(self, value):
        self.model.embed_tokens = value
    
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        position_ids=None,
        past_key_values=None,
        inputs_embeds=None,
        labels=None,
        use_cache=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        return super().forward(
            input_ids, 
            attention_mask, 
            position_ids, 
            past_key_values, 
            inputs_embeds,
            labels,
            use_cache,
            output_attentions,
            output_hidden_states,
            return_dict
        )

In [10]:
model = Gemma2ForSequenceClassification.from_pretrained( #CustomGemma2ForSequenceClassification
    config.checkpoint,
    num_labels=config.num_labels,
    torch_dtype=torch.float16,
    device_map='auto',
)

config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/6.13G [00:00<?, ?B/s]

Some weights of Gemma2ForSequenceClassification were not initialized from the model checkpoint at unsloth/gemma-2-9b-it-bnb-4bit and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
model.config.use_cache = False
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

In [ ]:
train = pd.read_json('data/client_dataset.json')
train = train.fillna('нет информации')

In [13]:
def remove_duplicate_blocks(text: str) -> str:
    lines = text.split('\n')

    unique_blocks = set()
    result_lines = []
    
    for line in lines:
        if line not in unique_blocks:
            unique_blocks.add(line)
            result_lines.append(line)
    
    return '\n'.join(result_lines)

In [14]:
train.work_experience = train.work_experience.apply(remove_duplicate_blocks)

In [15]:
counts = train.grade_proof.value_counts().to_dict()
total_samples = sum(counts.values())
num_classes = len(counts)
class_weights = {cls: total_samples / (num_classes * count) for cls, count in counts.items()}

In [16]:
train, test = train_test_split(train, test_size=0.1, stratify=train['grade_proof'], random_state=52)

In [17]:
ds_train = Dataset.from_pandas(train)
ds_test = Dataset.from_pandas(test)

In [18]:
ds = Dataset.from_pandas(train)

In [19]:
class CustomTokenizer:
    def __init__(
        self, 
        tokenizer: PreTrainedTokenizerBase, 
        max_length: int
    ) -> None:
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __call__(self, batch: dict) -> dict:
        age = [f'\n\n<age>: {t}' for t in batch['age']]
        city = [f'\n\n<city>: {t}' for t in batch['city']]
        work_experience = [f'\n\n<work_experience>: {t}' for t in batch['work_experience']][:-700]
        position = [f'<position>: {t}' for t in batch['position']][:100]
        key_skills = [f'\n\n<key_skills>: {t}' for t in batch['key_skills']][:200]
        client_name = [f'\n\n<client_name>: {t}' for t in batch['client_name']]
        salary = [f'\n\n<salary>: {t}' for t in batch['salary']][:200]
        
        texts = [
            a + c + we + p + ks + cn + s
            for a, c, we, p, ks, cn, s in zip(
                age,
                city,
                work_experience,
                position,
                key_skills,
                client_name,
                salary
            )
        ]
            
        tokenized = self.tokenizer(texts, max_length=self.max_length, truncation=True)
        
        labels = []
        for grade_proof in batch['grade_proof']:
            if grade_proof == 'не подтверждён':
                label = 0
            elif grade_proof == 'подтверждён':
                label = 1
            labels.append(label)
            
        return {**tokenized, 'labels': labels}

In [20]:
# chris deott version, try to truncate from left side

class DeottCustomTokenizer:
    def __init__(
        self, 
        tokenizer: PreTrainedTokenizerBase, 
        max_length: int
    ) -> None:
        self.tokenizer = tokenizer
        self.max_length = max_length

    def prepare_text(self, age, city, work_experience, position, key_skills, client_name, salary):
        rounds = [
            f'<start_of_turn>age\n{age}\n\ncity\n{city}\n\n'
            + f'work_experience\n{work_experience[:-700]}\n\n'
            + f'position\n{position[:100]}\n\n'
            + f'key_skills\n{key_skills[:200]}\n\n'
            + f'client_name\n{client_name}\n\n'
            + f'salary\n{salary[:200]}<end_of_turn>'
        ]
        
        tmp = '\n'.join(rounds)
        for k in range(len(rounds)):
            tmp = '\n'.join(rounds[k:])
            if len(self.tokenizer(tmp)['input_ids'] ) < self.max_length: 
                break
        
        return tmp
        
    def __call__(self, batch: dict) -> dict:
        texts = [
            self.prepare_text(a, c, we, p, ks, cn, s)
            for a, c, we, p, ks, cn, s in zip(
                batch['age'], 
                batch['city'], 
                batch['work_experience'], 
                batch['position'], 
                batch['key_skills'], 
                batch['client_name'], 
                batch['salary'], 
            )
        ]
        
        tokenized = self.tokenizer(texts, max_length=self.max_length, truncation=True)
        
        labels = []
        for grade_proof in batch['grade_proof']:
            if grade_proof == 'не подтверждён':
                label = 0
            elif grade_proof == 'подтверждён':
                label = 1
            labels.append(label)
            
        return {**tokenized, 'labels': labels}

In [21]:
if USE_DEOTT_TOKENIZER:
    encode = DeottCustomTokenizer(tokenizer, max_length=config.max_length)
else:
    encode = CustomTokenizer(tokenizer, max_length=config.max_length)

In [22]:
ds_train = ds_train.map(encode, batched=True)
ds_test = ds_test.map(encode, batched=True)

Map:   0%|          | 0/11405 [00:00<?, ? examples/s]

Map:   0%|          | 0/1268 [00:00<?, ? examples/s]

In [23]:
class FocalLoss(nn.Module):
    '''
    Focal Loss, as described in https://arxiv.org/abs/1708.02002.

    It is essentially an enhancement to cross entropy loss and is
    useful for classification tasks when there is a large class imbalance.
    x is expected to contain raw, unnormalized scores for each class.
    y is expected to contain class labels.

    Shape:
        - x: (batch_size, C) or (batch_size, C, d1, d2, ..., dK), K > 0.
        - y: (batch_size,) or (batch_size, d1, d2, ..., dK), K > 0.
    '''
    def __init__(
        self,
        alpha: torch.Tensor = None,
        gamma: float = 0.,
        reduction: str = 'mean',
        ignore_index: int = -100
    ):
        '''
        Constructor.

        Args:
            alpha (Tensor, optional): Weights for each class. Defaults to None.
            gamma (float, optional): A constant, as described in the paper.
                Defaults to 0.
            reduction (str, optional): 'mean', 'sum' or 'none'.
                Defaults to 'mean'.
            ignore_index (int, optional): class label to ignore.
                Defaults to -100.
        '''
        if reduction not in ('mean', 'sum', 'none'):
            raise ValueError('Reduction must be one of: "mean", "sum", "none".')

        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.ignore_index = ignore_index
        self.reduction = reduction
        self.nll_loss = nn.NLLLoss(
            weight=alpha, reduction='none', ignore_index=ignore_index
        )

    def forward(self, x: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
        if x.ndim > 2:
            # (N, C, d1, d2, ..., dK) --> (N * d1 * ... * dK, C)
            c = x.shape[1]
            x = x.permute(0, *range(2, x.ndim), 1).reshape(-1, c)
            # (N, d1, d2, ..., dK) --> (N * d1 * ... * dK,)
            y = y.view(-1)

        unignored_mask = y != self.ignore_index
        y = y[unignored_mask]
        if len(y) == 0:
            return torch.tensor(0., device=x.device, dtype=x.dtype)
        x = x[unignored_mask]

        # compute weighted cross entropy term: -alpha * log(pt)
        # (alpha is already part of self.nll_loss)
        log_p = F.log_softmax(x, dim=-1)
        ce = self.nll_loss(log_p, y)

        # get true class column from each row
        all_rows = torch.arange(len(x))
        log_pt = log_p[all_rows, y]

        # compute focal term: (1 - pt)^gamma
        pt = log_pt.exp()
        focal_term = (1 - pt) ** self.gamma

        # the full loss: -alpha * ((1 - pt)^gamma) * log(pt)
        loss = focal_term * ce

        if self.reduction == 'mean':
            loss = loss.mean()
        elif self.reduction == 'sum':
            loss = loss.sum()

        return loss

In [24]:
class FocalTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        if class_weights is not None:
            if isinstance(class_weights, list):
                class_weights = torch.tensor(class_weights, dtype=torch.float)
            self.class_weights = class_weights.cuda() if torch.cuda.is_available() else class_weights
        else:
            self.class_weights = None

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get('labels')
        outputs = model(**inputs)
        logits = outputs.get('logits')

        loss_foc = FocalLoss(alpha=self.class_weights, gamma=2)
        loss = loss_foc(logits.squeeze(), labels.squeeze())
        return (loss, outputs) if return_outputs else loss

In [25]:
class FocalTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        if class_weights is not None:
            if isinstance(class_weights, list):
                class_weights = torch.tensor(class_weights, dtype=torch.float)
            self.class_weights = class_weights.cuda() if torch.cuda.is_available() else class_weights
        else:
            self.class_weights = None

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get('labels')
        outputs = model(**inputs)
        logits = outputs.get('logits')

        loss_foc = FocalLoss(alpha=self.class_weights, gamma=2)
        loss = loss_foc(logits.squeeze(), labels.squeeze())
        return (loss, outputs) if return_outputs else loss

In [26]:
class TimeLimitCallback(TrainerCallback):
    def __init__(self, time_limit_hours):
        self.time_limit_seconds = time_limit_hours * 3600
        self.start_time = None

    def on_train_begin(self, args, state, control, **kwargs):
        self.start_time = time.time()

    def on_step_end(self, args, state, control, **kwargs):
        elapsed_time = time.time() - self.start_time
        if elapsed_time >= self.time_limit_seconds:
            control.should_training_stop = True

In [27]:
def compute_metrics(eval_preds: EvalPrediction) -> dict:
    y_true = eval_preds.label_ids
    y_pred = eval_preds.predictions.argmax(-1)
    y_proba = eval_preds.predictions[:, 1]

    return {
        'roc_auc': roc_auc_score(y_true=y_true, y_score=y_proba),
        'f1 (minor class)': f1_score(y_true=y_true, y_pred=y_pred),  
    }

In [28]:
def get_optimizer(model, head_LR, alpha, rank):
    backbone_LR = alpha / rank * head_LR
    
    backbone_params = list(model.base_model.model.model.parameters())
    head_params = list(model.score.parameters())
    
    optimizer = AdamW8bit([
        {'params': backbone_params, 'lr': backbone_LR},
        {'params': head_params, 'lr': head_LR},
    ])
    
    return optimizer

In [29]:
optimizer = get_optimizer(model, head_LR=config.lr, alpha=config.lora_alpha, rank=config.lora_r)

In [30]:
num_training_steps = (
    len(ds_train) // training_args.per_device_train_batch_size
) * training_args.num_train_epochs

num_warmup_steps = int(0.05 * num_training_steps)

scheduler = get_scheduler(
    'cosine', optimizer=optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps
)

In [31]:
trainer = FocalTrainer(
    args=training_args, 
    model=model,
    optimizers=(optimizer, scheduler),
    tokenizer=tokenizer,
    train_dataset=ds_train,
    eval_dataset=ds_test,
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    class_weights=list(class_weights.values()),
    callbacks=[TimeLimitCallback(time_limit_hours=11)]
)

In [32]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: lightsource- (lightsource-unk). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.3
wandb: Run data is saved locally in /kaggle/working/wandb/run-20241130_121157-hjwkdcay
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run output-l4-2nd-try
wandb: ⭐️ View project at https://wandb.ai/lightsource-unk/FIC%20Gemma2-9b
wandb: 🚀 View run at https://wandb.ai/lightsource-unk/FIC%20Gemma2-9b/runs/hjwkdcay
/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we wil

Epoch,Training Loss,Validation Loss,Roc Auc,F1 (minor class)
0,0.024900,0.323482,0.593430,0.000000


/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an except

TrainOutput(global_step=1425, training_loss=0.8448501523050311, metrics={'train_runtime': 35234.2422, 'train_samples_per_second': 0.324, 'train_steps_per_second': 0.04, 'total_flos': 4.863941975950848e+17, 'train_loss': 0.8448501523050311, 'epoch': 0.9994739610731194})